In [6]:
import numpy as np
import matplotlib.pyplot as plt
import cv2


# empty, full = cv2.imread('empty.png'), cv2.imread('full.png') 


cutout_size = {'x':(630,1000), 'y':(500,1100)} 
cutout = lambda img: img[cutout_size['x'][0]:cutout_size['x'][1], cutout_size['y'][0]:cutout_size['y'][1]]

def show(img):
    cv2.imshow('img', img) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def analyse_frame(frame):
    # cut out important part


    img = cutout(frame)
    # convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    threshold = 220
    # apply threshold
    gray = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)[1]


    # find contours
    contours, hierarchy = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # find largest contour
    contour = max(contours, key=cv2.contourArea)

    # find bounding rectangle
    x, y, w, h = cv2.boundingRect(contour)

    full = w > 450


    # draw contours
    img = cv2.drawContours(img, contours, -1, (100, 0, 0), 1)

    color = (0, 255, 0) if full else (0, 0, 255)

    img = cv2.drawContours(img, contour, -1, color, 4)


    # draw bounding rectangle
    # if full:
    #     img = cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    # else:
    #     img = cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)


    area = cv2.contourArea(contour) / 10000

    txt = "Filter capacity good"
    txt_col = (255,100,100)
    if not full:
        txt = "Roll empty!"
        txt_col = (0,0,255)
    else:
        if area < 1.5:
            txt = "Running low"
            txt_col = (0,255,255)
        
    img = cv2.fillPoly(img, pts =[contour], color=txt_col)

    # draw text of contour area

    # place cutout back in frame
    frame[cutout_size['x'][0]:cutout_size['x'][1], cutout_size['y'][0]:cutout_size['y'][1]] = img

    cv2.putText(frame, txt, (300, 300), cv2.FONT_HERSHEY_SIMPLEX, 4, (0,0,0), 10) 
    cv2.putText(frame, txt, (300, 300), cv2.FONT_HERSHEY_SIMPLEX, 4, txt_col, 4) 


    return frame


def analyse_video(path):
    cap = cv2.VideoCapture(path)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = analyse_frame(frame)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


# write to video
def write_video(path):
    cap = cv2.VideoCapture(path)

    # fps = cap.get(cv2.CAP_PROP_FPS)
    # width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    # height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fourcc =cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter('short_output_quick.avi', fourcc, int(24), (int(cap.get(3)*2),int(cap.get(4))))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        orig = frame.copy()
        frame = analyse_frame(frame)
        # cv2.imshow('frame', frame)
        tot = np.concatenate((orig, frame), axis=1)
        out.write(tot)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [7]:
write_video("combined_filter_roll-short.mp4")